# Decision Analysis Project 1

## Installs and Imports

In [ ]:
#Installs go here

In [ ]:
#Imports go here

## Utilities

In [1]:
class Criteria:
    '''
    Criteria class for PROMETHEE and ELECTRE implementations.
    '''
    def __init__(self, name: str, criteria_type: str, w: float, q: float, p:float, v:float = float("inf")):
        '''
        Initialize instance of Criteria.
        Parameters:
            - name (str): Criteria name, e.g. (cost, completion time, rating, etc.)
            - criteria_type (str): Criteria type - 'gain' (higher values preffered) or 'cost' (lower values preffered)
            - w (float): Criteria weight
            - q (float): Indifference threshold
            - p (float): Perference threshold
            - v (float, optional): Veto threshold -> used only for ELECTRE
        '''
        self.name = name
        assert criteria_type.lower() == "gain" or criteria_type.lower() == "cost", "Invalid criteria type"
        self.criteria_type = criteria_type.lower()
        self.w = w
        self.q = q
        self.p = p
        self.v = v


    def __str__(self):
        '''
        Print information about Criteria.
        '''
        print(f"Criteria: {self.name} \n type: {self.criteria_type} \n w: {self.w} \n q: {self.q} \n p: {self.p} \n v: {self.v} \n")

## PROMETHEE

## SRF

## ELECTRE

## Use